In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
class RNN(nn.Module):

    def __init__(self, size):
        super(RNN, self).__init__()
        self.W1 = nn.Linear(size, size)
        self.W2 = nn.Linear(size, size, bias=False)
        self.h0 = nn.Parameter(torch.Tensor(size), 
                               requires_grad=True)

    def forward(self, xs):
        hs = [self.h0]
        for i in range(xs.size(0)):
            h = nn.Tanh()(self.W1(hs[i]) + self.W2(xs[i]))
            hs.append(h)
        return hs

In [3]:
rnn = RNN(5)
a = Variable(torch.Tensor(100, 5).fill_(1), requires_grad=True)
hs = rnn(a)
loss = hs[100].sum()
loss[0].backward()
print(hs[100].grad)


None


In [33]:
class Gated(nn.Module):
    def __init__(self, size):
        super(RNN, self).__init__()
        self.W1 = nn.Linear(size, size)
        self.W2 = nn.Linear(size, size, bias=False)
        self.activation = nn.Tanh()

        self.gate_W1 = nn.Linear(size, size)
        self.gate_W2 = nn.Linear(size, size, bias=False)
        self.gate_activation = nn.Sigmoid()
        self.h0 = nn.Parameter(torch.Tensor(size))

    def forward(self, xs):
        hs = [self.h0]
        for i in range(xs.size(0)):
            h = self.activation(self.W1(hs[i]) + self.W2(xs[i]))
            t = self.gate_activation(self.gate_W1(hs[i]) + self.gate_W2(xs[i]))
            
            hs.append((1-t) * h[i] + t * h )
        return hs